In [5]:
import pandas as pd
import numpy as np
import pickle

In [6]:
data = pd.read_csv('customer_churn.csv')
data.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
data = data.drop(['customer_id'], axis=1)

In [8]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [16]:
data['gender'] = label_encoder.fit_transform(data['gender'])

In [10]:
## One-Hot Encoding for 'Country' column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse_output=False)
onehot_encoded_country = onehot_encoder.fit_transform(data[['country']])

In [11]:
onehot_encoded_country

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], shape=(10000, 3))

In [12]:
df = pd.DataFrame(onehot_encoded_country, columns=onehot_encoder.get_feature_names_out(['country']))

In [ ]:
# 'data' is already defined in previous cells, so no need to redefine or import pandas again
data = pd.concat([data.drop(['country'], axis=1), df], axis=1)


,credit_score,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,country_France,country_Germany,country_Spain
0,619,Female,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,Female,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,Female,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,Female,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,Female,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [17]:
data.head()

,credit_score,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,country_France,country_Germany,country_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [18]:
## Save the encoders and scaler
with open('label_encoded_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder, file) 

with open('onehot_encoded_country.pkl', 'wb') as file:
    pickle.dump(onehot_encoder, file)

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [20]:
X = data.drop(['churn'], axis = 1)
y = data['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)   

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [24]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

c:\Users\LOQ\anaconda3\envs\myenv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

In [26]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [27]:
early_stopping_callbacks = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [30]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping_callbacks, tensorflow_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8026 - loss: 0.4540 - val_accuracy: 0.8355 - val_loss: 0.3877
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8466 - loss: 0.3795 - val_accuracy: 0.8585 - val_loss: 0.3534
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8522 - loss: 0.3543 - val_accuracy: 0.8570 - val_loss: 0.3452
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8570 - loss: 0.3451 - val_accuracy: 0.8595 - val_loss: 0.3426
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8600 - loss: 0.3398 - val_accuracy: 0.8560 - val_loss: 0.3489
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8622 - loss: 0.3358 - val_accuracy: 0.8550 - val_loss: 0.3406
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8631 - loss: 0.3324 - val_accuracy: 0.8580 - val_loss: 0.3427
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8639 - loss: 0.3299 - val_accu

In [31]:
model.save('churn_model.h5')

In [33]:
%load_ext tensorboard
%tensorboard --logdir logs/fit/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
